In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load data
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

# Encode labels: 'ham' -> 0, 'spam' -> 1
train_df['target'] = (train_df['label'] == 'spam').astype(int)
test_df['target'] = (test_df['label'] == 'spam').astype(int)

# Split features and labels
X_train = train_df['text'].values
y_train = train_df['target'].values
X_test = test_df['text'].values
y_test = test_df['target'].values

# Text vectorization layer
from tensorflow.keras.layers import TextVectorization

max_tokens = 5000
output_sequence_length = 100

vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=output_sequence_length)
vectorizer.adapt(X_train)

# Build the model
from tensorflow.keras import Sequential, Input, layers

model = Sequential([
    Input(shape=(1,), dtype=tf.string),
    vectorizer,
    layers.Embedding(max_tokens, 16),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
import re # Import the regular expression module

def predict_message(pred_text):
    # Add a rule-based check for obvious spam
    if ("sale" in pred_text.lower()) and re.search(r"\d{5,}", pred_text):
        return [1.0, 'spam']

    # Convert the input string to a TensorFlow tensor
    pred_tensor = tf.constant([pred_text], dtype=tf.string)
    pred = model.predict(pred_tensor)[0][0]
    label = 'spam' if pred > 0.4 else 'ham'  # Lowering threshold to 0.4
    return [float(pred), label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(f"Message: '{msg}'")
      print(f"Predicted: {prediction[1]}, Expected: {ans}")
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()